This notebook contains function for generating the three probability models. There is some error handling with all of them. The parameters alpha, beta, and p_max have defaults, but they will have to be tuned further, probably with a sensitivity analysis.

In [ ]:
import numpy as np

In [ ]:
# function for inverse distance, alpha modifies the stretch, and values returned range from 0 to p_max
def true_distance_probabilities(distances, alpha=1, p_max=0.5):
    # Convert to numpy
    distances = np.array(distances)

    # Error handling
    if distances.shape[0] != distances.shape[1]:
        return("ERROR: Distance Matrix not square")
    if np.min(distances < 0):
        print("ERROR: Negative Distance")
    if p_max < 0 or p_max > 1:
        return("ERROR: Invalid value of p_max")

    # Function
    new_distances = (distances < 1).astype(int) + distances # replace everything less than 1 with 1
    result = np.power(np.power(new_distances, alpha), -1)

    # Rescale array
    arr_min = np.min(result)
    arr_max = np.max(result)
    result = ((result - arr_min)/(arr_max - arr_min)) * p_max

    return result

In [ ]:
# function for gravity model, alpha modifies the stretch, and values returned range from 0 to p_max
def gravity_model(distances, populations, alpha=0.1, p_max=0.5):
    # Convert to numpy
    distances = np.array(distances)
    populations = np.array(populations)

    # Error handling
    if distances.shape[0] != distances.shape[1]:
        return("ERROR: Distance Matrix not square")
    if np.min(distances < 0):
        print("ERROR: Negative Distance")
    if distances.shape[0] != populations.shape[0]:
        return("ERROR: Distance and Populations are not the same size")
    if np.min(populations) < 0:
        print("ERROR: Negative Population")
    if p_max < 0 or p_max > 1:
        return("ERROR: Invalid value of p_max")

    # Function
    populations = populations / 1000 # prevent integer overflow
    new_distances = (distances < 1).astype(int) + distances # replace everything less than 1 with 1
    weight = np.multiply(np.tile(np.atleast_2d(populations).T, (1, len(populations))), np.tile(populations, (len(populations), 1)))
    result = np.power(np.divide(weight, new_distances), alpha)

    # Rescale array
    arr_min = np.min(result)
    arr_max = np.max(result)
    result = ((result - arr_min)/(arr_max - arr_min)) * p_max
    
    return result

In [ ]:
# function for gravity model, alpha modifies the stretch for attractiveness, beta modifies the stretch for distance, and values returned range from 0 to p_max
def huff_model(attractiveness, distances, alpha=0.1, beta=0.1, p_max=0.5):
    # Convert to numpy
    distances = np.array(distances)
    attractiveness = np.array(attractiveness)

    # Error handling
    if distances.shape[0] != distances.shape[1]:
        return("ERROR: Distance Matrix not square")
    if np.min(distances < 0):
        print("ERROR: Negative Distance")
    if distances.shape[0] != attractiveness.shape[0]:
        return("ERROR: Distance and Populations are not the same size")
    if np.min(attractiveness) < 0 or np.max(attractiveness) > 1:
        print("ERROR: Attractivness not on scale [0,1]")
    if p_max < 0 or p_max > 1:
        return("ERROR: Invalid value of p_max")

    # Function
    new_distances = (distances < 1).astype(int) + distances # replace everything less than 1 with 1
    A_j_alpha = np.power(np.tile(np.atleast_2d(attractiveness).T, (1,len(attractiveness))), alpha)
    D_i_j_beta = np.power(new_distances,-1*beta)
    denominator = np.sum(new_distances, axis=1) * np.sum(attractiveness)
    result = np.divide(np.multiply(A_j_alpha, D_i_j_beta), denominator)

    # Rescale array
    arr_min = np.min(result)
    arr_max = np.max(result)
    result = ((result - arr_min)/(arr_max - arr_min)) * p_max

    return result